In [4]:
# Directly accessing gmail api

import os.path

# !pip3 install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly"]

def gmail_authenticate():
    creds = None
    # The file token.json stores the user's access and refresh tokens, and is created
    # automatically when the authorization flow completes for the first time.
    if os.path.exists("token.json"):
        print('using exisiting token')
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            print("token expired - authentication required")
            creds.refresh(Request())
        else:
            print("authentication required")
            flow = InstalledAppFlow.from_client_secrets_file(
                "credentials.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    # return the gmail service
    return build("gmail", "v1", credentials=creds)

# get the Gmail API service
service = gmail_authenticate()

def get_messages(query):
    try:
        # Call the Gmail API
        result = service.users().messages().list(userId='me',q=query).execute()
        messages = [ ]
        if 'messages' in result:
            messages.extend(result['messages'])
        while 'nextPageToken' in result:
            page_token = result['nextPageToken']
            result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
            if 'messages' in result:
                messages.extend(result['messages'])
        return messages

    except HttpError as error:
        # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")

# all messages
messages = get_messages(query='')

using exisiting token
token expired - authentication required


In [1]:
# Using simplegmail abstraction

# !pip3 install simplegmail
from simplegmail import Gmail

gmail = Gmail()

# list lables
gmail.list_labels()

# Unread messages in your inbox
# messages = gmail.get_unread_inbox()

# Starred messages
messages = gmail.get_starred_messages()

# ...and many more easy to use functions can be found in gmail.py!

# Print them out!
for message in messages:
    # print("To: " + message.recipient)
    print("From: " + message.sender)
    print("Date: " + message.date)
    print("\tSubject: " + message.subject)
    # print("Preview: " + message.snippet)
    
    # print("Message Body: " + message.plain)  # or message.html


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=395092168133-6pck9snola6m5gqtp8mt8ciau26fdncp.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.settings.basic&access_type=offline&response_type=code&prompt=consent

If your browser is on a different machine then exit and re-run this
application with the command-line parameter

  --noauth_local_webserver

Authentication successful.
From: Google Maps Timeline <noreply-maps-timeline@google.com>
Date: 2024-08-21 01:35:11-04:00
	Subject: 🌎 A new device is saving visits in Timeline
From: Jennifer Leale <jleale@fsu.edu>
Date: 2024-08-09 09:16:54-04:00
	Subject: Interview for the Application Developer/Designer Position
From: Google <google-noreply@google.com>
Date: 2024-07-28 11:52:25-04:00
	Subject: Your Gmail storage is 82% full


In [24]:
# Get mails with query
from simplegmail import query

messages = gmail.get_messages(query=query.construct_query(newer_than=(11, "day"), labels=[["INBOX"]]))

In [ ]:
print(f'size={len(messages)}')
for message in messages:
    print(f'{message}, {message.date}')
    print(f'\t{message.subject}')
    print(f'\t{message.snippet}')
    # print(message.plain)

In [ ]:
message = service.users().messages().get(
                userId='me', id='1913749036c02855'
            ).execute()
message